# Intro to tensorflow

In [1068]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [1069]:
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)
sess = tf.Session()
output = sess.run(add)
print(output) # 12
sess.close()

12


In [1070]:
c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c, d)
sess = tf.Session()
output = sess.run(add, feed_dict={c:5, d:7})
print(output) # 12
output = sess.run(add, feed_dict={c:20, d:32})
print(output) # 52
sess.close()


12
52


### NOTE:
Following tensorflow 2 from now on.

# Logistic regress on Tensorflow

In [1071]:
import numpy as np
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [1072]:
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

In [1073]:
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

In [1074]:
y = tf.sigmoid(tf.matmul(x, W) + b)
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

In [1075]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [1076]:
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [1077]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [1078]:
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
# Display the correct answer rate every 100 times
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


In [1079]:
#Check if the learning result is correct
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})
#Check output y
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})
print(classified)
# [[ True]
# [ True]
# [ True]
# [ True]]

print(prob)
# [[  1.96514215e-04]
# [  4.90498319e-02]
# [  4.90498319e-02]
# [  9.31203783e-01]]

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9654632e-04]
 [4.9049824e-02]
 [4.9049824e-02]
 [9.3120384e-01]]


In [1080]:
print('W:', sess.run(W))
print('b:', sess.run(b))
# W: [[ 5.5699544]
# [ 5.5699544]]
# b: [-8.53457928]

W: [[5.5699544]
 [5.5699544]]
b: [-8.534579]


In [1081]:
sess.close()

# Tensorflow (v1)

# Problem 1
Looking back on the scratch

### Steps to build a ML model
1. Load
2. Define model (class) along with needed stuffs (params, input shapes,..., procedure, batch ...)
3. Init stuff and use the model

# Problem 2
Correspond between scratch and tensorflow

In [1082]:
"""
Binary classification of Iris dataset using neural network implemented in TensorFlow
"""
import pandas as pd
from sklearn.model_selection import train_test_split

#Load dataset
df = pd.read_csv("Iris.csv")

#Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# Convert label to number
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    Iterator to get a mini-batch
    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output
#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})            
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples

        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))


Epoch 0, loss : 1.7062, val_loss : 4.3175, acc : 0.562
Epoch 1, loss : 0.9475, val_loss : 3.3044, acc : 0.562
Epoch 2, loss : 0.6002, val_loss : 7.3488, acc : 0.375
Epoch 3, loss : 0.3332, val_loss : 2.4291, acc : 0.625
Epoch 4, loss : 0.3623, val_loss : 4.0366, acc : 0.500
Epoch 5, loss : 0.1949, val_loss : 1.1861, acc : 0.688
Epoch 6, loss : 0.1861, val_loss : 2.2722, acc : 0.562
Epoch 7, loss : 0.1023, val_loss : 0.8318, acc : 0.812
Epoch 8, loss : 0.1148, val_loss : 1.6800, acc : 0.562
Epoch 9, loss : 0.0733, val_loss : 0.6209, acc : 0.812
test_acc : 0.800


### Steps to build in tensorflow
1. Load dataset
2. Define model and it's flow
   1. Define parameters (variable) and inputs (placeholders)
   2. Define model's prediction (or forward flow)
   3. Define models' loss function
   4. Get an optimizer
   5. Define the traing flow
      - Optimize
      - Training scores
      - Loss
      - Log,...
3. Prepare epoch and batch
4. Initialize paremeters
5.  Start a session
  - Train in batches
  - Use the model and stuss defined in tf syntax
6.  Close session

**Tensorflow 1.0 has a lazy execution approach to it.**

# Problem 3
Create a model of Iris using all three types of objective variables

## NOTE
**Let's go through the process one by one and test them all!**

## Data Prep

In [1083]:
#load dataset
df = pd.read_csv('Iris.csv')

print('shape: ', df.shape)
print(df.head(5))

shape:  (150, 6)
   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa


In [1084]:
# predictor
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
print('X shape:', X.shape)
print(X.head(5))

X shape: (150, 4)
   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0            5.1           3.5            1.4           0.2
1            4.9           3.0            1.4           0.2
2            4.7           3.2            1.3           0.2
3            4.6           3.1            1.5           0.2
4            5.0           3.6            1.4           0.2


In [1085]:
# label
y = df['Species']
print('y shape: ', y.shape)
print(y.head(5))

y shape:  (150,)
0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Species, dtype: object


In [1086]:
# Convert to numpy
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(150, 4) (150,)


In [1087]:
# Encode y from text to number label
print('init shape y: ', y.shape)
#reshape for onehot use
y = y.reshape(-1,1)
#label encode
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y).toarray()
print('shape: ', y.shape)
print(y[:5])

init shape y:  (150,)
shape:  (150, 3)
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [1088]:
from sklearn.model_selection import train_test_split
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9)
print('Train/Test size: ', [data.shape for data in [X_train, X_test, y_train, y_test]])
# train validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = 0.8)
print('Train/Test size: ', [data.shape for data in [X_train, X_val, y_train, y_val]])

Train/Test size:  [(135, 4), (15, 4), (135, 3), (15, 3)]
Train/Test size:  [(108, 4), (27, 4), (108, 3), (27, 3)]


## Model Prep

In [1089]:
# some variables
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = y_train.shape[1]

print('input nodes: ', n_input)
print('sample size: ', n_samples)
print('prediction classes: ', n_classes)

input nodes:  4
sample size:  108
prediction classes:  3


In [1090]:
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

print('minibatch prepared!')

minibatch prepared!


In [1091]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100

print('HYPER PARAMS')
print('alpha learning rate: ', learning_rate)
print('batch size: ', batch_size)
print('epoch num', num_epochs)
print('n_hidden1: ', n_hidden1)
print('n_hidden2: ', n_hidden2)

HYPER PARAMS
alpha learning rate:  0.001
batch size:  10
epoch num 100
n_hidden1:  50
n_hidden2:  100


In [1092]:
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
print('X holder: ',X)
print('Y holder: ', Y)

X holder:  Tensor("Placeholder_276:0", shape=(None, 4), dtype=float32)
Y holder:  Tensor("Placeholder_277:0", shape=(None, 3), dtype=float32)


# Model

### Test layer

In [1093]:
# weight
w1 = tf.Variable(tf.random_normal([n_input, n_classes]))
b1 = tf.Variable(tf.random_normal([n_classes]))
print('w:', w1)
print('b:', b1)

w: <tf.Variable 'Variable_425:0' shape=(4, 3) dtype=float32>
b: <tf.Variable 'Variable_426:0' shape=(3,) dtype=float32>


In [1094]:
# layer
print('X: ', X)
layer1 = tf.matmul(X, w1)
print('after matrix multiplication:', layer1)
layer1 = tf.add(layer1, b1)
print('after adding bias:', layer1)
layer1 = tf.nn.relu(layer1)
print('after activation: ', layer1)

X:  Tensor("Placeholder_276:0", shape=(None, 4), dtype=float32)
after matrix multiplication: Tensor("MatMul_217:0", shape=(None, 3), dtype=float32)
after adding bias: Tensor("Add_306:0", shape=(None, 3), dtype=float32)
after activation:  Tensor("Relu_122:0", shape=(None, 3), dtype=float32)


In [1095]:
def do(stuff, session, input):
  return session.run(stuff, feed_dict = input)
def prep_session():
  session = tf.Session()
  session.run(tf.global_variables_initializer())
  return session


with prep_session() as s:
  forward = do(layer1, s, {X: X_train, Y: y_train})
  print('forward shape: ', forward.shape)
  print('forward: ', forward[0])


forward shape:  (108, 3)
forward:  [0.5547868 0.        0.       ]


## Objective Function

In [1096]:
logits = layer1 # assuming 1 layer model
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
print('loss:', loss_op)


# the equivalent or the tf.softmax's implementation!
softmax = tf.nn.softmax(logits)
loss_op2 = -tf.reduce_sum(Y * tf.log(softmax), 1)
loss_op2 = tf.reduce_mean(loss_op2)

loss: Tensor("Mean_169:0", shape=(), dtype=float32)


In [1097]:
# test loss op
tf.set_random_seed(0)
with prep_session() as s:
  x_in, y_in = X_train[:5], y_train[:5]
  run = lambda x: do(x, s, {X: x_in, Y: y_in})
  print('y_true: ', y_in)

  loss = run(loss_op)
  print('loss: ', loss)
  loss2 = run(loss_op2)
  print('loss2: ', loss2.mean())

y_true:  [[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
loss:  1.2593744
loss2:  1.2593744


## Prediction

In [1099]:
# TODO

## Optimizer

In [ ]:
# TODO

## Training

In [ ]:
# TODO